<a href="https://colab.research.google.com/github/raketic-ognjen/Crypto-Portfolio-Optimization/blob/main/Crypto_Portfolio_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font face="courier" size = 6 color = "Tangerine"><center>**Crypto Portfolio Optimization - Timespan 2020-2023**</center></font>

## <font face="courier" size = 4 color = "Tangerine"><center>**Core Idea**</center></font>

### I will test 3 crypto portfolios:

* 1st will be diversified:
    * Bitcoin (BTC)
    * Ethereum (ETH)
    * Binance Coin (BNB)
    * Cardano (ADA)
    * Solana (SOL)
    * Polkadot (DOT)
    * Chainlink (LINK)
    * Litecoin (LTC)
    * Ripple (XRP)
    * Stellar (XLM)

* 2nd will be focused on High Risk, High Reward:
   * Dogecoin (DOGE)
   * Shiba Inu (SHIB)
   * SafeMoon (SAFEMOON)
   * Polygon (MATIC)
   * VeChain (VET)
   * Theta (THETA)
   * Filecoin (FIL)
   * Aave (AAVE)
   * Decentraland (MANA)
   * SushiSwap (SUSHI)

* 3rd will be focused on NFT and DeFi:
    * Uniswap (UNI)
    * Compound (COMP)
    * Chainlink (LINK)
    * Aave (AAVE)
    * Maker (MKR)
    * Decentraland (MANA)
    * Enjin Coin (ENJ)
    * Flow (FLOW)
    * Axie Infinity (AXS)
    * The Sandbox (SAND)

I will try to test this 3 portfolio to see their frontiers and CAPM Models.

In [58]:
#Python Libraries
import yfinance as yf
import numpy as np
import numpy.linalg as LA
import scipy as sc
import scipy.optimize as sco
import scipy.stats as stat
import pandas as pd
import pandas_datareader as pdr
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sklearn as skl
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import keras as k
import sympy as sp
import statsmodels.api as sm
import seaborn as sns

### Function Expected Return

In [55]:
def expRet(ret, n=252, com=False):
  """Calculates expected returns in one of two methods. Parameters are
    - ret: Pandas' DataFrame or Series of returns
    - n: optional, number of compounding periods in a year (252 for days, 52 for weeks, 12 for months, 4 for quarters...)
    - com: optional, deterimines whether expected returns are calculated as annualized sample mean or annualized compunded return"""
  if com:
    return (1+ret).prod()**(n/len(ret))-1
  else:
    return ret.mean()*n

### Expected Return using Principal Component Function

In [59]:
def erPCA(ret,rf,f=252,n=None):
  if n!=None and ret.shape[1]<n:
    return print('Wrong input. Desired number of component n has to be smaller than or equal to number of stocks')
  else:
    l,evec=np.linalg.eig(ret.cov()*f)
    a=(evec[:,:np.count_nonzero(l>l.mean())] if n==None else evec[:,:n])
    rets_pca = ret@a
    errf,erpca=rf.mean(),rets_pca.mean()
    return[f*(errf+sm.OLS(ret[c]-rf,rets_pca).fit().params@erpca) for c in ret.columns]

### Annualized volatility function

In [60]:
def annualize_vol(ret,n=252):
  """Calculates volatility of sample returns. Parameters are:
    - ret: Pandas' DataFrame or Series of returns
    - n: optional, number of compounding periods in a year (252 for days, 52 for weeks, 12 for months, 4 for quarters...)"""
  return  ret.std()*(n**0.5)

### Expected Returns on Portfolio function

In [62]:
def per(W,ret=None, n=252,com=False,er_assumed=None):
  """Calculates expected returns of portfolio:
    - ret: data set of returns
    - W: iterable of weights
    - n: optional, number of compounding periods in a year (252 for days, 52 for weeks, 12 for months, 4 for quarters...)
    - com: optional, deterimines whether expected returns are calculated as annualized sample mean or annualized compunded retu
    - er_assumed: optional, assummed expected returns on each stock"""
  er = expRet(ret,n,com) if np.all(er_assumed==None) else er_assumed
  return er@W

### Variance on portfolio function

In [63]:
def pV(W,ret=None,n=252,vol=False,cov_assumed=None):
  """Calculates variance returns of portfoli:
    - ret: data set of returns
    - W: iterable of weights
    - n: optional, number of compounding periods in a year (252 for days, 52 for weeks, 12 for months, 4 for quarters...)
    - vol: optional, determines whether you want to display portfolio's volatility (True) or variance (False)
    - cov_assumed: optional, assummed annualized covariance matrix of returns"""
  CovM=(ret.cov()*n if np.all(cov_assumed==None) else cov_assumed)
  return np.sqrt(W@CovM@W) if vol else W@CovM@W

### MVP function

In [64]:
def mvp(ret=None,f=252,com=False, cov_assumed=None):
  """Calculates MVP portfolio weights, volatility and expected returns for given data set of returns. Paramters are:
    - ret: optional, data set of returns. If it isn't provided, than it is required to provide er_assumed and cov_assumed.
    - f: optional, number of compounding periods in a year, i.e. frequency (252 for days, 52 for weeks, 12 for months, 4 for quarters...)
    - com: optional, deterimines whether expected returns are calculated as annualized sample mean or annualized compunded ret
    - cov_assumed: optional, assummed annualized covariance matrix of returns"""
  n=(len(ret.columns) if np.all(ret!=None) else len(cov_assumed))
  result=sco.minimize(lambda w: pV(w,ret,f,False,cov_assumed),[1/n]*n,constraints=[dict(type='eq',fun=lambda w:sum(w)-1)])
  return dict(w=result.x,er=per(result.x,ret,f,com),vol=np.sqrt(result.fun)) if np.all(ret!=None) else result.x

### Optimal portfolio weights, volatility and expected returns Function

In [65]:
def targetP(ret=None, mi=None,Bounds=None,f=252,com=False,er_assumed=None,cov_assumed=None):
  """Calculates optimal portfolio weights, volatility and expected returns. Paramters are:
    - ret: optional, data set of returns. If it isn't provided, than it is required to provide er_assumed and cov_assumed.
    - mi: target level of expected returns. If mi is omitted, function will caulculate MVP weights!
    - f: optional, number of compounding periods in a year, i.e. frequency (252 for days, 52 for weeks, 12 for months, 4 for quarters...)
    - com: optional, deterimines whether expected returns are calculated as annualized sample mean or annualized compunded retu
    - er_assumed: optional, assummed expected returns on each stock
    - cov_assumed: optional, assummed annualized covariance matrix of returns
    - Bounds: optional, simple list of length 2 (so that elements represent upper and lower bound) or complex list of length n
    (so that each sublist represent collection of upper and lower bound for each of n stocks separately) """
  n=(len(ret.columns) if np.all(ret!=None) else len(cov_assumed))
  if mi==None:
    result=sco.minimize(lambda w: pV(w,ret,f,False,cov_assumed),[1/n]*n,constraints=[dict(type='eq',fun=lambda w:sum(w)-1)])
    if np.all(ret!=None):
      return dict(w=result.x, er=per(result.x,ret,f,com,er_assumed),vol=np.sqrt(result.fun))
    else:
      return dict(w=result.x,er=(None if np.all(er_assumed==None) else (result.x)@er_assumed),vol=np.sqrt(result.fun))
  elif type(mi)==float or type(mi)==np.float64:
    result= sco.minimize(lambda w: pV(w,ret,f,False,cov_assumed),[1/n]*n,
                         bounds=(None if Bounds== None else [Bounds]*n if len(Bounds)==2 else Bounds),
                    constraints=[dict(type='eq',fun=lambda w:sum(w)-1),
                                 dict(type='eq',fun=lambda w:per(w,ret,f,com,er_assumed)-mi)])
    return dict(w=result.x, er=mi, vol=np.sqrt(result.fun))
  else:
    return print('\33[91mWrong input given for parameter mi!\33[0m')

### Effective Frontier function

In [66]:
def EF(ret=None,Range=[0.01,0.4],plot=None,f=252,com=False,er_assumed=None,cov_assumed=None):
    """Prepares data for plotting efficient frontier or plot this curve (depending on what user chooses). Paramters are:
    - ret: optional, data set of returns. If it isn't provided, than it is required to provide er_assumed and cov_assumed.
    - Range: optional, range in which you want to see efficient frontier.
    - f: optional, number of compounding periods in a year, i.e. frequency (252 for days, 52 for weeks, 12 for months, 4 for quarters...)
    - com: optional, deterimines whether expected returns are calculated as annualized sample mean or annualized compunded retu
    - plot: optional, determines whether you want to plot efficient frontier or not. Possible values are:
        a) None - on values will be displayed
        b) 'curve' - only curve will be plotted
        c) 'mvp' - curve with MVP will be plotted
        d) 'full' - curve with MVP will be plotted and efficient part will be emphasised
    - er_assumed: optional, assummed expected returns on each stock
    - cov_assumed: optional, assummed annualized covariance matrix of returns"""
    mi=np.arange(Range[0],Range[1]+0.01,0.01)
    sigma=np.array([targetP(ret,m,f=f,com=com,er_assumed=er_assumed,cov_assumed=cov_assumed)['vol'] for m in mi])
    if plot==None:
        return dict(sigma=sigma,er=mi)
    elif plot=='curve':
        fig=go.Figure()
        fig.add_trace(go.Scatter(x=sigma,y=mi,line=dict(color='Blue',width=3)))
        fig.update_layout(xaxis=dict(title_text='\$\sigma$',range=[-0.01,max(0.25,np.max(sigma)+0.01)],zerolinecolor='Black'),
                  yaxis=dict(title_text='$\mu$',zerolinecolor='Black'),
                  title=dict(text="Efficient frontier",x=0.5,y=0.87,font=dict(size=25,color='Navy')))
        return fig
    elif plot=='mvp':
        M=targetP(ret,m,f=f,com=com,er_assumed=er_assumed,cov_assumed=cov_assumed)
        fig=go.Figure()
        fig.add_trace(go.Scatter(x=sigma,y=mi,line=dict(color='Blue',width=3)))
        fig.add_trace(go.Scatter(x=[M['vol']],y=[M['er']],marker=dict(color='Red',line=dict(color='Black',width=2),size=10)))
        fig.update_layout(xaxis=dict(title_text='\$\sigma$',range=[-0.01,max(0.25,np.max(sigma)+0.01)],zerolinecolor='Black'),
                  yaxis=dict(title_text='$\mu$',zerolinecolor='Black'),showlegend=False,
                  title=dict(text="Efficient frontier",x=0.5,y=0.87,font=dict(size=25,color='Navy')))
        return fig
    elif plot=='full':
        M=targetP(ret,f=f,com=com,er_assumed=er_assumed,cov_assumed=cov_assumed)
        fig=go.Figure()
        fig.add_trace(go.Scatter(x=sigma,y=mi,line=dict(color='Blue',dash='dash'),fill="tozeroy",fillcolor='Pink'))
        fig.add_trace(go.Scatter(x=sigma[mi>M['er']],y=mi[mi>M['er']],line=dict(color='Blue',width=3)))
        fig.add_trace(go.Scatter(x=[M['vol']],y=[M['er']],marker=dict(color='Red',line=dict(color='Black',width=2),size=10)))
        fig.update_layout(xaxis=dict(title_text='\$\sigma$',range=[-0.01,max(0.25,np.max(sigma)+0.01)],zerolinecolor='Black'),
                  yaxis=dict(title_text='$\mu$',zerolinecolor='Black'),showlegend=False,
                  title=dict(text="Efficient frontier",x=0.5,y=0.87,font=dict(size=25,color='Navy')))

        return fig
    else:
        return print("\33[91mWrong input ginven for parameter plot. Expected values are: None,'curve','mvp' or 'full'\33[0m")

### Max Sharpe Function

In [68]:
def maxSharpe(ret=None,rf=0.0,Bounds=[-1,1],f=252,com=False,er_assumed=None,cov_assumed=None):
    """Calculates weights for portfolio with maximal Sharpe's ratio, its volatility and expected returns. Paramters are:
    - ret: optional, data set of returns on risky assets. If it isn't provided, than it is required to provide er_assumed and cov_assumed.
    - f: optional, number of compounding periods in a year, i.e. frequency (252 for days, 52 for weeks, 12 for months, 4 for quarters...)
    - com: optional, deterimines whether expected returns are calculated as annualized sample mean or annualized compunded retu
    - rf: data set of returns on risk-free assets or assumed constant
    - er_assumed: optional, assummed expected returns on each stock
    - cov_assumed: optional, assummed annualized covariance matrix of returns"""
    n=(len(ret.columns) if np.all(ret!=None) else len(cov_assumed))
    r_f=(rf if type(rf)==float or type(rf)==np.float64 else expRet(rf))
    result=sco.minimize(lambda w: -(per(w,ret,f,com,er_assumed)-r_f)/pV(w,ret,f,True,cov_assumed),[1/n]*n,bounds=[Bounds]*n,
                        constraints=[dict(type='eq',fun=lambda w:sum(w)-1)])
    return dict(w=result.x, er=per(result.x,ret,f,com,er_assumed),vol=pV(result.x,ret,f,True,cov_assumed),sharpe=-result.fun)

### Portfolio tracking error Function

In [67]:
def portfolio_tracking_error(weights,ref_r,bb_r):
  return np.sqrt(((ref_r-(weights*bb_r).sum(axis=1))**2).sum())

### Style Analysis Function

In [69]:
def style_analysis(dependent_variable, explanatory_variables):
    """Returns the optimal weights that minimizes the Tracking error between a portfolio of the explanatory variables and the
    dependent variable. Parameters are:
    1. dependent_variable - data set of target returns
    2. explanatory_variables - data set of explanatory returns
    All parameters should be given as np.array, pd.Series or pd.DataFrame."""
    n=len(explanatory_variables.columns)
    result=sco.minimize(lambda w: portfolio_tracking_error(w,dependent_variable, explanatory_variables),
                        [1/n]*n,bounds=[(0,1)]*n,constraints=[dict(type='eq',fun=lambda w:sum(w)-1)])
    return pd.Series(result.x, index=explanatory_variables.columns)

### Cumulative simple returns Function

In [70]:
def compound(r):
    """returns the result of compounding the set of returns in r"""
    return np.expm1(np.log1p(r).sum())

### Implied Returns Function

In [71]:
def implied_returns(delta, sigma, w):
    """Obtain the implied expected returns by the market - i.e. construct vector Pi. Parameters are:
    - delta: Risk Aversion Coefficient (scalar)
    - sigma: Variance-Covariance Matrix (N x N) as pd.DataFrame
    - w: Portfolio weights (N x 1) as pd.Series
    Returns an N x 1 vector of Returns as pd.Series"""
    ir = delta * sigma.dot(w).squeeze() # to get a series from a 1-column dataframe
    ir.name = 'Implied Returns'
    return ir

### Proportional Prior function

In [73]:
def proportional_prior(sigma, tau, p):
    """Returns the He-Litterman simplified Omega. Parameters are:
    - sigma: N x N Covariance Matrix as pd.DataFrame
    - tau: a scalar
    - p: a K x N pd.DataFrame linking Q and Assets
    returns a P x P pd.DataFrame, a Matrix representing Prior Uncertainties"""
    helit_omega = p.dot(tau * sigma).dot(p.T)
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index=p.index,columns=p.index)

### Black-Litterman model Function

In [74]:
def bl(w_prior, sigma_prior, p, q,omega=None,delta=2.5, tau=.02, rf=0):
    """Computes the posterior expected returns based on the original black litterman reference model. Parameters are:
    - W_prior: must be an N x 1 vector of weights, a pd.Series
    - Sigma_prior: is an N x N covariance matrix, a pd.DataFrame
    - P: must be a K x N matrix linking Q and the Assets, a pd.DataFrame
    - Q: must be an K x 1 vector of views, a pd.Series
    - Omega: must be a K x K matrix a pd.DataFrame, or None (in that case He-Litterman simplified Omega is used)
    - delta and tau: are scalars
    - rf: optional, assumed risk-free rate. By default it is set to 0 """
    if omega is None: omega = proportional_prior(sigma_prior, tau, p) #if omega isn't specified use He-Litterman siplification
    N = w_prior.shape[0] # How many assets do we have?
    K = q.shape[0] # And how many views?
    pi = implied_returns(delta, sigma_prior,  w_prior) # Determine market implied expected returns - Pi
    sigma_prior_scaled = tau * sigma_prior  # Adjust (scale) Sigma by the uncertainty scaling factor
    # estimating expected returns and covariance matrix by B-L model
    mu_bl=rf+pi+sigma_prior_scaled.dot(p.T).dot(LA.inv(p.dot(sigma_prior_scaled).dot(p.T) + omega).dot(q - p.dot(pi).values))
    sigma_bl=sigma_prior+sigma_prior_scaled-sigma_prior_scaled.dot(p.T).dot(LA.inv(p.dot(sigma_prior_scaled).dot(p.T)+omega)).dot(p).dot(sigma_prior_scaled)
    return (mu_bl, sigma_bl)

### Optimal (Tangent/Max Sharpe Ratio) Portfolio weights by using the Markowitz Optimization Procedure function

In [75]:
def w_msr(sigma, mu,rf=0):
    """Optimal (Tangent/Max Sharpe Ratio) Portfolio weights by using the Markowitz Optimization Procedure. Parameters are:
    - Mu: the vector of Excess expected returns given as pd.Series
    - Sigma: covariance matrix. Must be given as N x N matrix in pd.DataFrame"""
    invS=pd.DataFrame(LA.inv(sigma.values), index=sigma.columns, columns=sigma.index)
    w = invS.dot(mu-rf)
    return w/sum(w)

### W-Star function

In [76]:
def w_star(delta, sigma, mu):
    return (pd.DataFrame(LA.inv(sigma.values), index=sigma.columns, columns=sigma.index).dot(mu))/delta

# 1st portfolio

In [42]:
crypto1 = ['BTC-USD', 'ETH-USD', 'BNB-USD', 'ADA-USD', 'SOL-USD', 'DOT-USD', 'LINK-USD', 'LTC-USD', 'XRP-USD', 'XLM-USD']

In [43]:
#Yf download
cryptoprice1 = yf.download(crypto1, start='2021-4-16')['Adj Close']
cryptoprice1.head()

[*********************100%%**********************]  10 of 10 completed


,ADA-USD,BNB-USD,BTC-USD,DOT-USD,ETH-USD,LINK-USD,LTC-USD,SOL-USD,XLM-USD,XRP-USD
Date,,,,,,,,,,
2021-04-16,1.416949,510.220184,61572.789062,41.537975,2431.946533,42.101555,310.612732,25.397366,0.613263,1.557527
2021-04-17,1.384869,520.416748,60683.820312,42.535950,2344.895020,40.372066,304.759918,25.099874,0.599255,1.560055
2021-04-18,1.275501,481.034241,56216.183594,37.347431,2237.136963,39.065910,273.098816,32.302917,0.544734,1.405998
2021-04-19,1.197959,505.092926,55724.265625,34.816669,2166.188721,35.879383,261.919220,31.675896,0.501234,1.315044
2021-04-20,1.265934,587.029114,56473.031250,35.444435,2330.210938,38.950634,260.941345,31.770054,0.527950,1.383299


In [44]:
rets1 = cryptoprice1.pct_change().dropna()
rets1

,ADA-USD,BNB-USD,BTC-USD,DOT-USD,ETH-USD,LINK-USD,LTC-USD,SOL-USD,XLM-USD,XRP-USD
Date,,,,,,,,,,
2021-04-17,-0.022640,0.019985,-0.014438,0.024026,-0.035795,-0.041079,-0.018843,-0.011713,-0.022842,0.001623
2021-04-18,-0.078974,-0.075675,-0.073622,-0.121980,-0.045954,-0.032353,-0.103889,0.286975,-0.090981,-0.098751
2021-04-19,-0.060793,0.050014,-0.008750,-0.067763,-0.031714,-0.081568,-0.040936,-0.019411,-0.079856,-0.064690
2021-04-20,0.056742,0.162220,0.013437,0.018031,0.075719,0.085599,-0.003733,0.002973,0.053300,0.051903
2021-04-21,-0.046763,-0.069853,-0.045454,-0.044391,0.014823,-0.064550,-0.008185,0.023386,-0.067927,-0.062749
...,...,...,...,...,...,...,...,...,...,...
2023-09-27,-0.002461,-0.001851,0.005167,-0.005823,0.002557,0.037794,-0.004261,0.005611,0.005713,-0.004208
2023-09-28,0.017684,0.014579,0.025380,0.021032,0.034674,0.023607,0.027596,0.043262,0.015176,0.019872
2023-09-29,0.001670,0.000946,-0.004064,0.000638,0.009112,0.016861,0.006028,0.017188,-0.011831,0.023932


In [57]:
#Weights for portfolio 1
w1 = [1/10]*10

#portfolio returns
cryptorp = rets1@w1
cryptorp

Date
2021-04-17   -0.012172
2021-04-18   -0.043520
2021-04-19   -0.040547
2021-04-20    0.051619
2021-04-21   -0.037166
                ...   
2023-09-27    0.003824
2023-09-28    0.024286
2023-09-29    0.006048
2023-09-30    0.009935
2023-10-01    0.006547
Length: 898, dtype: float64

In [77]:
exr1 = expRet(rets1,252,True)
print(exr1)
print("*"*50)

anvol1 = annualize_vol(rets1,252)
print(anvol1)

ADA-USD    -0.378776
BNB-USD    -0.216479
BTC-USD    -0.205951
DOT-USD    -0.474603
ETH-USD    -0.099705
LINK-USD   -0.376140
LTC-USD    -0.351477
SOL-USD    -0.031503
XLM-USD    -0.379135
XRP-USD    -0.266573
dtype: float64
**************************************************
ADA-USD     0.771742
BNB-USD     0.659085
BTC-USD     0.521562
DOT-USD     0.842896
ETH-USD     0.684278
LINK-USD    0.868736
LTC-USD     0.764630
SOL-USD     1.022864
XLM-USD     0.778431
XRP-USD     0.865533
dtype: float64


In [78]:
#Portfolio expected return
crypto1per = per(W=w1,ret=rets1,com=True)
crypto1per

-0.27803427396139657

In [80]:
#pV
cryptopV1 = pV(w1,rets1,252,True)
cryptopV1

0.6717547850142082

In [81]:
#MVP
crypto1mvp = mvp(rets1,252,True)
crypto1mvp

{'w': array([ 0.03202802,  0.29464423,  1.1360983 , -0.14151706, -0.10997085,
        -0.12821297, -0.08902768, -0.04036183,  0.09581789, -0.04949806]),
 'er': -0.17411128558018876,
 'vol': 0.49484337531125006}

In [85]:
#Manual optimization 1 - Only with pV
crypto1optres = sco.minimize(lambda w1: pV(w1,rets1),
                             [1/10]*10,
                             constraints=[dict(type='eq',fun=lambda w1: sum(w1)-1)])
crypto1optres

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.24486996608943068
       x: [ 3.203e-02  2.946e-01  1.136e+00 -1.415e-01 -1.100e-01
           -1.282e-01 -8.903e-02 -4.036e-02  9.582e-02 -4.950e-02]
     nit: 12
     jac: [ 4.897e-01  4.892e-01  4.899e-01  4.898e-01  4.899e-01
            4.895e-01  4.900e-01  4.897e-01  4.899e-01  4.895e-01]
    nfev: 133
    njev: 12

In [86]:
#Target of Portfolio Return
crypto1target = targetP(rets1,0.4)
crypto1target

{'w': array([-0.12963434,  0.39764044,  0.85024277, -0.62165904,  0.5080931 ,
        -0.14548845, -0.20064435,  0.26975897, -0.21102562,  0.28271652]),
 'er': 0.4,
 'vol': 0.6273505319628557}

In [88]:
#Manual optimization 2 - with pV and per
crypto1optres1 = sco.minimize(lambda w1: pV(w1,rets1),
                              [1/10]*10,
                              constraints=[dict(type='eq',fun=lambda w1:sum(w1)-1),
                                           dict(type='eq',fun=lambda w1:per(w1,rets1)-0.4)])
crypto1optres1

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.39356868995407807
       x: [-1.296e-01  3.976e-01  8.502e-01 -6.217e-01  5.081e-01
           -1.455e-01 -2.006e-01  2.698e-01 -2.110e-01  2.827e-01]
     nit: 12
     jac: [ 4.489e-01  5.407e-01  4.992e-01  3.900e-01  6.303e-01
            5.024e-01  4.752e-01  8.437e-01  4.469e-01  5.774e-01]
    nfev: 132
    njev: 12

In [90]:
#Manual optimization 3 -with pV and per and bounds
crypto1optres2 = sco.minimize(lambda w1: pV(w1,rets1),
                              [1/10]*10,
                              bounds=[[0,1.5]]*10,
                              constraints=[dict(type='eq',fun=lambda w1:sum(w1)-1),
                                           dict(type='eq',fun=lambda w1:per(w1,rets1)-0.4)])
crypto1optres2

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.7901691676370404
       x: [ 0.000e+00  1.709e-16  0.000e+00  0.000e+00  2.659e-01
            4.717e-18  1.584e-16  7.341e-01  0.000e+00  0.000e+00]
     nit: 3
     jac: [ 9.553e-01  8.358e-01  6.481e-01  1.098e+00  9.756e-01
            1.102e+00  9.397e-01  1.799e+00  8.591e-01  9.162e-01]
    nfev: 33
    njev: 3

In [95]:
#Effective Frontier without range
crypto1EF1 = EF(rets1,plot='full')
crypto1EF1

In [96]:
# With range
crypto1EF2 = EF(rets1,Range=[0.01,1.5],plot='full')
crypto1EF2

In [99]:
#Optimization for Sharpe's Ratio

crypto1opt = maxSharpe(rets1,0.02,com=True)
crypto1opt

{'w': array([-0.71507823,  1.        ,  1.        , -1.        ,  1.        ,
        -0.76664919, -0.51827257,  1.        , -1.        ,  1.        ]),
 'er': 0.7749092039709669,
 'vol': 1.245646515991798,
 'sharpe': 0.6060380647955327}

In [102]:
#Test for Efficient frontier

#initial Wealth
wealth1 = 30_000

##Determined amount invested in each stock
si1= crypto1opt['w']*wealth1

#Number of stocks
n1=si1/cryptoprice1.iloc[0]
n1

ADA-USD    -15139.815551
BNB-USD        58.798144
BTC-USD         0.487228
DOT-USD      -722.230676
ETH-USD        12.335798
LINK-USD     -546.285663
LTC-USD       -50.056471
SOL-USD      1181.224876
XLM-USD    -48918.652296
XRP-USD     19261.304002
Name: 2021-04-16 00:00:00, dtype: float64

In [103]:
#Value of portfolio each point in the time
ST1 = rets1@n1
ST1

Date
2021-04-17    1484.341279
2021-04-18    4189.157114
2021-04-19    3655.977421
2021-04-20   -2512.339358
2021-04-21    2913.653837
                 ...     
2023-09-27    -332.937706
2023-09-28    -604.413384
2023-09-29    1024.938513
2023-09-30    -243.459191
2023-10-01    -185.560261
Length: 898, dtype: float64

In [128]:
#preparing data
crypto1return1 = cryptoprice1.iloc[:,:10].pct_change()
crypto1return1.dropna(inplace=True)

#initial wealth
v1=30_000

#Optimal weights
crypto1mvp = mvp(crypto1return1)['w']
crypto1P40= targetP(crypto1return1,0.40)['w']

In [129]:
crypto1mvp

array([ 0.03202802,  0.29464423,  1.1360983 , -0.14151706, -0.10997085,
       -0.12821297, -0.08902768, -0.04036183,  0.09581789, -0.04949806])

In [130]:
crypto1P40

array([-0.12963434,  0.39764044,  0.85024277, -0.62165904,  0.5080931 ,
       -0.14548845, -0.20064435,  0.26975897, -0.21102562,  0.28271652])

In [131]:
#Vector of money invested into each asset
vcrypto1mvp1 = crypto1mvp*v
vcrypto1P401 = crypto1P40*v

In [132]:
vcrypto1mvp1

array([  960.84071686,  8839.32688107, 34082.94906734, -4245.51193911,
       -3299.12537893, -3846.38909585, -2670.83041416, -1210.85477374,
        2874.53677529, -1484.94183878])

In [133]:
vcrypto1P401

array([ -3889.03034086,  11929.21334492,  25507.28301976, -18649.77123685,
        15242.79299792,  -4364.653419  ,  -6019.33050233,   8092.76916725,
        -6330.76872732,   8481.49569651])

In [134]:
#Vector of innitial stock prices
crypto1p0= cryptoprice1.iloc[0,:10]
crypto1p0

ADA-USD         1.416949
BNB-USD       510.220184
BTC-USD     61572.789062
DOT-USD        41.537975
ETH-USD      2431.946533
LINK-USD       42.101555
LTC-USD       310.612732
SOL-USD        25.397366
XLM-USD         0.613263
XRP-USD         1.557527
Name: 2021-04-16 00:00:00, dtype: float64

In [135]:
#Number of shares
ncrypto1mvp1 = vcrypto1mvp1/crypto1P40
ncrypto1P401 = vcrypto1P401/crypto1P40

In [136]:
ncrypto1mvp1

array([ -7411.93021898,  22229.44621449,  40086.13819152,   6829.32549444,
        -6493.15131298,  26437.76304736,  13311.2664928 ,  -4488.6543112 ,
       -13621.74278876,  -5252.40556117])

In [137]:
ncrypto1P401

array([30000., 30000., 30000., 30000., 30000., 30000., 30000., 30000.,
       30000., 30000.])

In [138]:
#the number of shares that we need to hold over time, we can calculate the value of our portfolio on each date in testing sample simply by multiplying previous two
Vcrypto1mvp1 = cryptoprice1.iloc[:,:10]@ncrypto1mvp1
Vcrypto1P401 = cryptoprice1.iloc[:,:10]@ncrypto1P401

In [139]:
figst = go.Figure()

figst.add_trace(
    go.Scatter(x=cryptoprice1.index,
               y=Vcrypto1mvp1,
               line=dict(color='Blue',width=3),
               name='MVP Portfolio')
)
figst.add_trace(
    go.Scatter(x=cryptoprice1.index,
               y=Vcrypto1P401,
               line=dict(color='Crimson',width=3),
               name='Target 40% pa')
)
figst.update_layout(
    xaxis=dict(title_text='Date',zerolinecolor='Black'),
    yaxis=dict(title_text='Dollars',zerolinecolor='Black'),
    title=dict(text='Comparing 2 static strategies',
               x=0.5,
               y=0.87,
               font=dict(size=25,color='Navy'))
)

figst.show()

# 2nd portfolio

In [45]:
crypto2 = ['DOGE-USD', 'SHIB-USD', 'SAFEMOON-USD', 'MATIC-USD', 'VET-USD','THETA-USD','FIL-USD','AAVE-USD', 'MANA-USD','SUSHI-USD']
cryptoprice2 = yf.download(crypto2, start='2021-4-16')['Adj Close']
cryptoprice2.head()

[*********************100%%**********************]  10 of 10 completed


,AAVE-USD,DOGE-USD,FIL-USD,MANA-USD,MATIC-USD,SAFEMOON-USD,SHIB-USD,SUSHI-USD,THETA-USD,VET-USD
Date,,,,,,,,,,
2021-04-16,442.194397,0.365870,183.253845,1.313581,0.436447,0.000002,1.000000e-06,16.315783,14.282880,0.238089
2021-04-17,430.174683,0.284173,182.688660,1.526686,0.406253,0.000002,2.000000e-06,16.020266,13.407032,0.227585
2021-04-18,382.075104,0.320475,156.309326,1.349810,0.362901,0.000003,2.000000e-06,14.037149,11.756942,0.254632
2021-04-19,346.812561,0.407318,154.857834,1.317240,0.327200,0.000007,4.000000e-06,12.199388,10.820083,0.229339
2021-04-20,361.608185,0.319500,153.701645,1.396516,0.341524,0.000011,2.000000e-06,12.706245,11.742767,0.251997


In [46]:
rets2 = cryptoprice2.pct_change().dropna()
rets2

,AAVE-USD,DOGE-USD,FIL-USD,MANA-USD,MATIC-USD,SAFEMOON-USD,SHIB-USD,SUSHI-USD,THETA-USD,VET-USD
Date,,,,,,,,,,
2021-04-17,-0.027182,-0.223295,-0.003084,0.162232,-0.069181,0.000000,1.000000,-0.018112,-0.061322,-0.044118
2021-04-18,-0.111814,0.127746,-0.144395,-0.115856,-0.106712,0.500000,0.000000,-0.123788,-0.123076,0.118843
2021-04-19,-0.092292,0.270982,-0.009286,-0.024129,-0.098377,1.333333,1.000000,-0.130921,-0.079686,-0.099332
2021-04-20,0.042662,-0.215601,-0.007466,0.060183,0.043778,0.571429,-0.500000,0.041548,0.085275,0.098797
2021-04-21,-0.020657,-0.039358,-0.035288,-0.042022,0.051841,-0.454545,-0.500000,-0.016427,-0.057964,-0.098279
...,...,...,...,...,...,...,...,...,...,...
2022-10-03,0.040677,0.018486,0.027471,0.020623,0.049138,-1.000000,0.000000,0.052170,0.034510,0.042358
2022-10-05,-0.002861,-0.018602,-0.002908,-0.001390,0.018217,inf,-0.083333,0.119478,0.005376,-0.003089
2022-10-06,-0.015525,-0.019943,-0.017878,-0.011663,-0.021507,-1.000000,0.000000,0.029411,-0.012330,-0.019594


# 3rd portfolio

In [47]:
crypto3 = ['UNI-USD', 'COMP', 'LINK-USD', 'AAVE-USD', 'MKR-USD', 'MANA-USD', 'ENJ-USD', 'FLOW-USD', 'AXS-USD', 'SAND']

cryptoprice3 = yf.download(crypto3, start='2021-4-16')['Adj Close']
cryptoprice3.head()

[*********************100%%**********************]  10 of 10 completed


,AAVE-USD,AXS-USD,COMP,ENJ-USD,FLOW-USD,LINK-USD,MANA-USD,MKR-USD,SAND,UNI-USD
Date,,,,,,,,,,
2021-04-16,442.194397,7.448085,18.030001,3.027652,36.581551,42.101555,1.313581,3381.328857,7.543566,0.001272
2021-04-17,430.174683,7.817194,NaN,2.999848,35.695862,40.372066,1.526686,3341.222900,NaN,0.001257
2021-04-18,382.075104,6.650603,NaN,2.694819,32.571568,39.065910,1.349810,3233.734863,NaN,0.000698
2021-04-19,346.812561,8.106076,17.780001,2.412772,32.903790,35.879383,1.317240,3520.074951,7.435519,0.001361
2021-04-20,361.608185,8.564796,17.219999,2.565713,34.380268,38.950634,1.396516,3452.383301,7.474809,0.001369


In [49]:
rets3 = cryptoprice3.pct_change().dropna()
rets3

,AAVE-USD,AXS-USD,COMP,ENJ-USD,FLOW-USD,LINK-USD,MANA-USD,MKR-USD,SAND,UNI-USD
Date,,,,,,,,,,
2021-04-17,-0.027182,0.049558,0.000000,-0.009183,-0.024211,-0.041079,0.162232,-0.011861,0.000000,-0.011792
2021-04-18,-0.111814,-0.149234,0.000000,-0.101681,-0.087525,-0.032353,-0.115856,-0.032170,0.000000,-0.444710
2021-04-19,-0.092292,0.218848,-0.013866,-0.104663,0.010200,-0.081568,-0.024129,0.088548,-0.014323,0.949857
2021-04-20,0.042662,0.056590,-0.031496,0.063388,0.044873,0.085599,0.060183,-0.019230,0.005284,0.005878
2021-04-21,-0.020657,-0.046243,0.027875,-0.032196,-0.027676,-0.064550,-0.042022,0.164467,0.034166,0.220599
...,...,...,...,...,...,...,...,...,...,...
2023-09-27,-0.007562,-0.016085,0.024476,0.001373,-0.002415,0.037794,-0.011732,0.049176,-0.008529,0.006369
2023-09-28,0.084424,0.016647,0.040956,0.006004,0.015275,0.023607,0.019330,0.011382,0.017204,0.025316
2023-09-29,0.005542,0.018550,-0.049180,0.003544,0.019065,0.016861,0.056888,-0.040891,-0.014799,0.000000
